<a href="https://colab.research.google.com/github/hardikkamboj/Google-Earth-Engine-Notebooks/blob/main/USE_CASES/Analyzing_cyclone_winston/Sentinal_image_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd 
import datetime as dt


In [ ]:
# this dataframe stores the information for dates 
df = pd.read_csv('dataframe.csv')


In [2]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=huzOBxHWjwYP9aIlvV-nyiet07lq6TImS2X6-R2RCNg&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWj_8pmCoxaEOeKOfCVXz7bO5jUxz-tnTPFfUdCl6kM9HxB67EHRwoM

Successfully saved authorization token.


In [3]:
!pip install --quiet wxee

     |████████████████████████████████| 43 kB 725 kB/s 
     |████████████████████████████████| 19.3 MB 1.2 MB/s 


In [4]:
import wxee
wxee.Initialize()

In [19]:
dates = df["date"]

date_1 = dt.datetime.strptime(dates.loc[0], '%Y-%m-%d %H:%M:%S')
date_2 = dt.datetime.strptime(dates.loc[1], '%Y-%m-%d %H:%M:%S')

week = dt.timedelta(days=7)
one_week_prior = date_1 - week 
one_week_post = date_2 + week


In [20]:
S2_SR = ee.ImageCollection('COPERNICUS/S2_SR')

In [21]:
# filtering on the basis of date
filtered_collection = S2_SR.filterDate(one_week_prior, one_week_post)

In [23]:
def addNDVI(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

In [24]:
S2_NDVI = S2_SR.map(addNDVI)

In [27]:
print(S2_NDVI.size())

ee.Number({
  "functionInvocationValue": {
    "functionName": "Collection.size",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.map",
          "arguments": {
            "baseAlgorithm": {
              "functionDefinitionValue": {
                "argumentNames": [
                  "_MAPPING_VAR_0_0"
                ],
                "body": {
                  "functionInvocationValue": {
                    "functionName": "Image.addBands",
                    "arguments": {
                      "dstImg": {
                        "argumentReference": "_MAPPING_VAR_0_0"
                      },
                      "srcImg": {
                        "functionInvocationValue": {
                          "functionName": "Image.rename",
                          "arguments": {
                            "input": {
                              "functionInvocationValue": {
                                "func

In [28]:

# downloading the dataset 
# download as image 
# The file name to save
description = "sentinel_images"
# The coordinate reference system to use (NAD83 Albers CONUS)
crs = "EPSG:5070"
# Spatial resolution in CRS units (meters)
scale = 100
# The region to download the image within.
region = ee.Geometry.Polygon(
    [[[179.53077, -17.22298 ],
      [179.26503, -17.22298],
      [179.26503, -17.43011],
      [179.53077, -17.43011, ]]]
)

In [29]:
files = filtered_collection.wx.to_tif(
    out_dir="data",
    prefix="wx_",
    region=region,
    scale=scale,
    crs=crs
)

Requesting data: 0it [00:00, ?it/s]